In [10]:
import cv2

img = cv2.imread("../input/zalando-store-crawl/zalando/hoodies-female/10K21J006-C11@8.jpg")

img.shape

In [11]:
import matplotlib.pyplot as plt

plt.imshow(img)

In [12]:
dir = "../input/zalando-store-crawl/zalando"

In [13]:
import tensorflow as tf

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.,                                                                                                                   
                                                           rotation_range=0.2,
                                                           width_shift_range=0.2,
                                                           height_shift_range=0.2,
                                                           zoom_range = 0.2, 
                                                           horizontal_flip=True,
                                                           validation_split = 0.02
                                                            )

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.,
                                                           validation_split = 0.2
                                                           )

In [14]:
train_data = train_gen.flow_from_directory(dir,
                                           target_size = (224,224),
                                           class_mode = "categorical",
                                           seed = 42,
                                           classes = ["hoodies","hoodies-female","longsleeve","shirt","sweatshirt","sweatshirt-female"],
                                           subset = "training"
                                           )

test_data = test_gen.flow_from_directory(dir,
                                         target_size = (224,224),
                                         class_mode = "categorical",
                                         classes = ["hoodies","hoodies-female","longsleeve","shirt","sweatshirt","sweatshirt-female"],
                                         seed = 42,
                                         subset = "validation"
                                         )


In [15]:
labels = list(train_data.class_indices.keys())

labels

In [16]:
# visualizing the train dataset

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize = (16,16))

for i in range(18):
  image,label = train_data.next()

  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(labels[tf.argmax(label[i])])
  plt.axis("off")


In [17]:
# Building a CNN model 
import tensorflow as tf
from tensorflow.keras import layers
model  = tf.keras.Sequential([
                              
                              layers.Conv2D(filters= 64, kernel_size= 2, activation="relu", input_shape=(224,224,3)),
                              layers.MaxPooling2D(pool_size= 2),

                              layers.Conv2D(filters = 64, kernel_size= 2, activation= "relu"),
                              layers.MaxPooling2D(pool_size= 2),

                              layers.Conv2D(filters = 64, kernel_size= 2, activation= "relu"),
                              layers.MaxPooling2D(pool_size= 2),

                              layers.Flatten(),

                              layers.Dense(128, activation="relu"),
                              layers.Dropout(0.5),

                              layers.Dense(6, activation= "softmax")
])

In [18]:
model.summary()

In [19]:
# compiling the model

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [20]:
# fitting data to the model

model.fit(train_data,
          epochs = 5,
          steps_per_epoch = len(train_data),
          validation_data = test_data,
          validation_steps = len(test_data)
          )

In [21]:
model_evaluation = model.evaluate(test_data)

In [22]:
print(f"Model Accuracy: {model_evaluation[1] * 100 : 0.2f} %")

In [23]:
# visualizing the test data

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,16))

for i in range(18):
  image, label = test_data.next()

  model_pred = model.predict(image)

  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(f"Prediction: {labels[tf.argmax(model_pred[i])]} \nOriginal : {labels[tf.argmax(label[i])]}")
  plt.subplots_adjust(top= 1.25)
  plt.axis("off")

## **Building a ResNet Model**

In [24]:
# building a ResNet model
import tensorflow as tf

base_model = tf.keras.applications.ResNet50V2(include_top= False) 

base_model.trainable = False

In [25]:
# building a resnet Model

import tensorflow as tf
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(224,224,3))

x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(6, activation="softmax")(x)

res_model = tf.keras.Model(inputs, outputs)


In [26]:
# compiling the model

res_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
    )

In [27]:
# fitting data to the model

res_model.fit(
    train_data,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = len(test_data)
)

## **Tunning the Model**

In [37]:
base_model.trainable = True

for layer in base_model.layers[:-5]:
  base_model.trainable = False

# compiling the model
res_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
    )
# fitting data

res_model.fit(
    train_data,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = len(test_data)
)


In [38]:
res_model_evaluation = res_model.evaluate(test_data)

In [39]:
print(f"Resnet Model Accuracy: {res_model_evaluation[1] * 100 :0.2f} %")

In [40]:
# predicting and visualizing the test data

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,16))

for i in range(18):
  image,label = test_data.next()
  res_model_pred = res_model.predict(image)

  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(f"Prediction: {labels[tf.argmax(res_model_pred[i])]}\nOriginal: {labels[tf.argmax(label[i])]}")
  plt.subplots_adjust(top= 1.25)
  plt.axis("off")